In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Demo recommend DATN/Crop_recommendation.csv")
df.head(5)

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [ ]:
df_new = df[['N', 'P', 'K', 'ph', 'label']]
df_new

,N,P,K,ph,label
0,90,42,43,6.502985,rice
1,85,58,41,7.038096,rice
2,60,55,44,7.840207,rice
3,74,35,40,6.980401,rice
4,78,42,42,7.628473,rice
...,...,...,...,...,...
2195,107,34,32,6.780064,coffee
2196,99,15,27,6.086922,coffee
2197,118,33,30,6.362608,coffee
2198,117,32,34,6.758793,coffee


In [ ]:
df_new['label'].value_counts()

label
rice           100
maize          100
chickpea       100
kidneybeans    100
pigeonpeas     100
mothbeans      100
mungbean       100
blackgram      100
lentil         100
pomegranate    100
banana         100
mango          100
grapes         100
watermelon     100
muskmelon      100
apple          100
orange         100
papaya         100
coconut        100
cotton         100
jute           100
coffee         100
Name: count, dtype: int64

In [ ]:
df_new.dtypes

N          int64
P          int64
K          int64
ph       float64
label     object
dtype: object

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [ ]:
X = df_new[['N', 'P', 'K', 'ph']]
y = df_new['label']

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_scaler = pd.DataFrame(scaler.fit_transform(X), columns=['N', 'P', 'K', 'ph'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaler, y, test_size = 0.1, random_state = 42, stratify=y)

In [ ]:
X_train

,N,P,K,ph
1099,0.835714,0.542857,0.240,0.311475
292,0.278571,0.507143,0.355,0.720203
1604,0.142857,0.014286,0.020,0.563638
1990,0.728571,0.285714,0.055,0.589311
289,0.157143,0.392857,0.400,0.669832
...,...,...,...,...
2026,0.578571,0.250000,0.200,0.454560
1620,0.028571,0.057143,0.005,0.630821
1286,0.042857,0.957143,0.970,0.433297
1218,0.200000,0.835714,0.960,0.365739


In [ ]:
from sklearn.metrics import classification_report
from sklearn import metrics

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(criterion="entropy",random_state=42,max_depth=10)

model.fit(X_train,y_train)

predicted_values = model.predict(X_test)
x = metrics.accuracy_score(y_test, predicted_values)

x

0.7863636363636364

In [ ]:
import joblib

# Lưu mô hình vào file
joblib.dump(model, '/content/drive/MyDrive/Demo recommend DATN/decision_tree_model.pkl')


['/content/drive/MyDrive/Demo recommend DATN/decision_tree_model.pkl']

In [ ]:
# Để load lại mô hình từ file
loaded_model = joblib.load('/content/drive/MyDrive/Demo recommend DATN/decision_tree_model.pkl')

In [ ]:
loaded_model

DecisionTreeClassifier(criterion='entropy', max_depth=10, random_state=42)

In [ ]:
N = 104
P = 18
K = 30
ph = 7.2

test_data = np.array([[N, P, K, ph]])
prediction = loaded_model.predict(test_data)
prediction

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array(['banana'], dtype=object)

dùng newron network

In [ ]:
from tensorflow import keras
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential

In [ ]:
model_nn = keras.Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(22, activation='softmax')  # 11 activation='softmax'
])


In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.fit_transform(y_test)
# print("Mapping:")
# for label, number in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)):
#     print(f"{label} --> {number}")

In [ ]:
y_train_encoded

array([ 1,  3, 16, ...,  7,  7,  7])

In [ ]:
model_nn.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',#sparse_categorical_crossentropy binary_crossentropy
                  metrics=['accuracy'])

In [ ]:
history = model_nn.fit(X_train, y_train_encoded, epochs=50, validation_data=(X_test, y_test_encoded))

Epoch 1/50
62/62 [==============================] - 1s 8ms/step - loss: 1.3431 - accuracy: 0.4626 - val_loss: 1.0949 - val_accuracy: 0.6318
Epoch 2/50
62/62 [==============================] - 1s 9ms/step - loss: 1.3108 - accuracy: 0.4707 - val_loss: 1.0502 - val_accuracy: 0.6318
Epoch 3/50
62/62 [==============================] - 0s 8ms/step - loss: 1.2268 - accuracy: 0.4990 - val_loss: 1.0163 - val_accuracy: 0.6318
Epoch 4/50
62/62 [==============================] - 0s 6ms/step - loss: 1.2157 - accuracy: 0.5086 - val_loss: 0.9821 - val_accuracy: 0.6455
Epoch 5/50
62/62 [==============================] - 0s 5ms/step - loss: 1.1920 - accuracy: 0.5152 - val_loss: 0.9504 - val_accuracy: 0.6455
Epoch 6/50
62/62 [==============================] - 0s 6ms/step - loss: 1.1573 - accuracy: 0.5187 - val_loss: 0.9254 - val_accuracy: 0.6682
Epoch 7/50
62/62 [==============================] - 0s 5ms/step - loss: 1.1219 - accuracy: 0.5222 - val_loss: 0.9022 - val_accuracy: 0.6364
Epoch 8/50
62/62 [==

In [ ]:
new_data_scaled

array([[0.74285714, 0.09285714, 0.125     , 0.57465835]])

In [ ]:
N = 104
P = 18
K = 30
ph = 7.2

test_data = np.array([[N, P, K, ph]])
new_data_scaled = scaler.transform(test_data)
predictions = model.predict_proba(new_data_scaled)
predictions

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.]])


Nếu mô hình của bạn được huấn luyện để chỉ dự đoán một nhãn duy nhất cho mỗi mẫu dữ liệu (như là một bài toán phân loại đa lớp), và bạn muốn mô hình đưa ra dự đoán về xác suất của từng nhãn, bạn có thể sử dụng một kỹ thuật gọi là "one-vs-all" hoặc "one-vs-rest".

Để thực hiện điều này, bạn cần sử dụng một mô hình huấn luyện cho mỗi nhãn riêng biệt. Khi dự đoán, bạn sẽ sử dụng tất cả các mô hình này để đưa ra dự đoán về xác suất của từng nhãn cho một mẫu dữ liệu mớ

In [ ]:
# Initialize a list to store probabilities for each label
all_label_probabilities = []

# Loop through each label
for label_index in range(len(label_encoder.classes_)):
    # Create a copy of the original model
    new_model = Sequential([
        Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')  # Output layer with sigmoid activation for binary classification
    ])

    # Compile the new model
    new_model.compile(optimizer='adam',
                      loss='binary_crossentropy',  # Use binary crossentropy for binary classification
                      metrics=['accuracy'])

    # Train the new model only for the current label
    new_y_train = (y_train == label_index).astype(int)  # Convert labels to binary for the current label
    new_model.fit(X_train, new_y_train, epochs=30, batch_size=32, validation_data=(X_test, (y_test == label_index).astype(int)))

    # Predict probabilities for the current label
    label_probabilities = new_model.predict_proba(new_data_scaled)
    all_label_probabilities.append(label_probabilities)

# Combine probabilities for all labels
all_label_probabilities = np.array(all_label_probabilities)

# Print probabilities for all labels
print(all_label_probabilities)


Epoch 1/30
62/62 [==============================] - 3s 11ms/step - loss: 0.2251 - accuracy: 0.9722 - val_loss: 0.0097 - val_accuracy: 1.0000
Epoch 2/30
62/62 [==============================] - 0s 5ms/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.0014 - val_accuracy: 1.0000
Epoch 3/30
62/62 [==============================] - 0s 4ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 5.0189e-04 - val_accuracy: 1.0000
Epoch 4/30
62/62 [==============================] - 0s 4ms/step - loss: 5.2864e-04 - accuracy: 1.0000 - val_loss: 2.3235e-04 - val_accuracy: 1.0000
Epoch 5/30
62/62 [==============================] - 0s 4ms/step - loss: 2.8229e-04 - accuracy: 1.0000 - val_loss: 1.3546e-04 - val_accuracy: 1.0000
Epoch 6/30
62/62 [==============================] - 0s 4ms/step - loss: 2.1584e-04 - accuracy: 1.0000 - val_loss: 8.6800e-05 - val_accuracy: 1.0000
Epoch 7/30
62/62 [==============================] - 0s 4ms/step - loss: 1.4210e-04 - accuracy: 1.0000 - val_loss: 5.9011e-05 - val_

AttributeError: 'Sequential' object has no attribute 'predict_proba'